In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce

from tqdm import tqdm
from utils import reduce_memory

import warnings
warnings.filterwarnings('ignore')

In [2]:
input_data_dir = '../data/original'
save_data_dir = '../data/preprocessed'

In [3]:
sales = pd.read_csv(f'{input_data_dir}/sales_train_evaluation.csv')
sell_prices = pd.read_csv(f'{input_data_dir}/sell_prices.csv')
calendar = pd.read_csv(f'{input_data_dir}/calendar.csv')

# reduce memory usage
sales = reduce_memory(sales)
sell_prices = reduce_memory(sell_prices)
calendar = reduce_memory(calendar)

In [4]:
#
calendar = calendar[calendar['d'].apply(lambda x: int(x.split('_')[1]) <= 1941)]

# 
calendar.date = pd.to_datetime(calendar.date)
calendar['year_delta'] = calendar.year - 2011

# cyclic encodings
calendar['quarter_sin'] = np.sin(2 * np.pi * calendar.date.dt.quarter/4.0)
calendar['quarter_cos'] = np.cos(2 * np.pi * calendar.date.dt.quarter/4.0)
calendar['month_sin'] = np.sin(2 * np.pi * calendar.month/12.0)
calendar['month_cos'] = np.cos(2 * np.pi * calendar.month/12.0)
calendar['day_sin'] = np.sin(2 * np.pi * calendar.date.dt.day/calendar.date.dt.days_in_month)
calendar['day_cos'] = np.cos(2 * np.pi * calendar.date.dt.day/calendar.date.dt.days_in_month)
calendar['weekday_sin'] = np.sin(2 * np.pi * calendar.wday/7.0)
calendar['weekday_cos'] = np.cos(2 * np.pi * calendar.wday/7.0)

# event count
calendar['event_count'] = calendar[['event_name_1', 'event_name_2']].notna().sum(axis=1)

# event encodings
event_names = ['event_name_1', 'event_name_2']
event_names_enc = ['event_name_1_enc', 'event_name_2_enc']
calendar[event_names_enc] = calendar[event_names]
event_names_encoder = ce.OrdinalEncoder(cols=event_names_enc)
event_names_encoder.fit(calendar)
event_names_encoder.mapping[1]['mapping'] = event_names_encoder.mapping[0]['mapping']
calendar = event_names_encoder.transform(calendar)
for col in event_names_enc:
    calendar[col] = calendar[col] - 1

event_types = ['event_type_1', 'event_type_2']
event_types_enc = ['event_type_1_enc', 'event_type_2_enc']
calendar[event_types_enc] = calendar[event_types]
event_type_encoder = ce.OrdinalEncoder(cols=event_types_enc)
event_type_encoder.fit(calendar)
event_type_encoder.mapping[1]['mapping'] = event_type_encoder.mapping[0]['mapping']
calendar = event_type_encoder.transform(calendar)
for col in event_types_enc:
    calendar[col] = calendar[col] - 1

#
calendar_df = calendar[['wm_yr_wk', 'd', 'snap_CA', 'snap_TX', 'snap_WI', 'year_delta',
                        'quarter_sin', 'quarter_cos', 'month_sin', 'month_cos', 
                        'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'event_count']
                        + event_names_enc 
                        + event_types_enc]

In [5]:
# melt
sales = pd.melt(sales, id_vars=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_vars=['d_'+str(i) for i in range(1, 1942)], value_name='sales')

# merge
data_df = sales.merge(right=calendar_df[['d', 'wm_yr_wk']], on=['d'], how='left')

# id
data_df.insert(0, 'id', data_df['store_id'] + '_' + data_df['item_id'])

# out of stock
def calculate_out_of_stock(group):
    first_sale = group[group['sales'] > 0].index.min()
    group['out_of_stock'] = 0
    if not pd.isna(first_sale):
        group.loc[first_sale:, 'out_of_stock'] = ((group.loc[first_sale:, 'sales'] == 0).astype(int).groupby((group.loc[first_sale:, 'sales'] != 0).cumsum()).cumsum())
        group['out_of_stock'] = group['out_of_stock'].where(group['sales'] == 0, 0)    
    return group
data_df = data_df.groupby('id').apply(calculate_out_of_stock).reset_index(drop=True)

# diff
data_df['sales_diff'] = data_df.groupby('id')['sales'].diff().fillna(0)

# lag
data_df['sales_lag1'] = data_df.groupby('id')['sales'].shift(1).fillna(0)
data_df['sales_lag7'] = data_df.groupby('id')['sales'].shift(7).fillna(0)
data_df['sales_lag28'] = data_df.groupby('id')['sales'].shift(28).fillna(0)
                                                                         
# rolling
data_df['sales_rolling7'] = data_df.groupby('id')['sales'].rolling(window=7).mean().fillna(0).reset_index(drop=True)
data_df['sales_rolling28'] = data_df.groupby('id')['sales'].rolling(window=28).mean().fillna(0).reset_index(drop=True)

# rolling diff
data_df['sales_rolling7_diff'] = data_df.groupby('id')['sales'].rolling(window=7).mean().diff().fillna(0).reset_index(drop=True)
data_df['sales_rolling28_diff'] = data_df.groupby('id')['sales'].rolling(window=28).mean().diff().fillna(0).reset_index(drop=True)

In [6]:
# id
sell_prices.insert(0, 'id', sell_prices['store_id'] + '_' + sell_prices['item_id'])

# release week
release = sell_prices.groupby('id')['wm_yr_wk'].min().reset_index()
release['release'] = 1
sell_prices['release'] = 0
sell_prices.loc[sell_prices.set_index(['id', 'wm_yr_wk']).index.isin(release.set_index(['id', 'wm_yr_wk']).index), 'release'] = 1

# relative sell price in store by week
sell_prices['sell_price_in_store'] = sell_prices['sell_price'] / sell_prices.groupby(['store_id', 'wm_yr_wk'])['sell_price'].transform('mean')

# diff
sell_prices['sell_price_diff'] = sell_prices.groupby('id')['sell_price'].diff().fillna(0)

# lag
sell_prices['sell_price_lag'] = sell_prices.groupby('id')['sell_price'].shift(1).fillna(0)

# rolling
sell_prices['sell_price_rolling'] = sell_prices.groupby('id')['sell_price'].rolling(window=4).mean().fillna(0).reset_index(drop=True)

# rolling diff
sell_prices['sell_price_rolling_diff'] = sell_prices.groupby('id')['sell_price'].rolling(window=4).mean().diff().fillna(0).reset_index(drop=True)

In [7]:
# merge
data_df = data_df.merge(right=sell_prices[['id', 'item_id', 'store_id', 'wm_yr_wk', 'sell_price', 'release', 'sell_price_in_store', 'sell_price_diff', 'sell_price_lag', 'sell_price_rolling', 'sell_price_rolling_diff']], on=['id', 'item_id', 'store_id', 'wm_yr_wk'], how='left')

# accumulate after release day
def accumulate_after_release(group):
    group = group.reset_index(drop=True)
    release = group[group['release'] == 1].index.min()
    group.loc[release:, 'release'] = range(1, len(group) - release + 1)
    return group
data_df = data_df.groupby('id').apply(accumulate_after_release).reset_index(drop=True)

# fill nan(before release) with 0 
data_df = data_df.fillna(0)

In [8]:
#
del calendar_df['wm_yr_wk']
calendar_df.to_csv(f'{save_data_dir}/calendar_df.csv', index=False)

#
del data_df['id']
del data_df['wm_yr_wk']

In [9]:
levels = [
    [],                        # Level 1: Total
    ['state_id'],              # Level 2: State
    ['store_id'],              # Level 3: Store
    ['cat_id'],                # Level 4: Category
    ['dept_id'],               # Level 5: Department
    ['state_id', 'cat_id'],    # Level 6: State-Category
    ['state_id', 'dept_id'],   # Level 7: State-Department
    ['store_id', 'cat_id'],    # Level 8: Store-Category
    ['store_id', 'dept_id'],   # Level 9: Store-Department
    ['item_id'],               # Level 10: Item
    ['item_id', 'state_id'],   # Level 11: Item-State
    ['item_id', 'store_id']    # Level 12: Individual
]

agg_funcs = {
    'sales': [ 
        ('sales_sum', 'sum'), # 판매량 합계
        ('sales_mean', 'mean'), # 판매량 평균값
        ('sales_std', 'std'), # 판매량 표준편차
        ('sales_max', 'max'), # 판매량 최대값
        ('sales_min', 'min'), # 판매량 최소값
     ],
    'sales_diff': [('sales_diff_mean', 'mean')], # 판매량 변화량 평균값
    'sales_lag1': [('sales_lag1_mean', 'mean')], # 1일 전 판매량 평균값
    'sales_lag7': [('sales_lag7_mean', 'mean')], # 7일 전 판매량 평균값
    'sales_lag28': [('sales_lag28_mean', 'mean')], # 28일 전 판매량 평균값
    'sales_rolling7': [('sales_rolling7_mean', 'mean')], # 7일 판매량 이동평균 평균값
    'sales_rolling28': [('sales_rolling28_mean', 'mean')], # 28일 판매량 이동평균 평균값
    'sales_rolling7_diff': [('sales_rolling7_diff_mean', 'mean')], # 7일 판매량 이동평균 변화량 평균값
    'sales_rolling28_diff': [('sales_rolling28_diff_mean', 'mean')], # 28일 판매량 이동평균 변화량 평균값

    'release': [('release_mean', 'mean')], # 최초 판매량의 평균값
    'out_of_stock': [('out_of_stock_mean', 'mean')], # 재고 없음 평균값

    'sell_price': [
        ('sell_price_mean', 'mean'), # 판매가격 평균값
        ('sell_price_std', 'std'), # 판매가격 표준편차
        ('sell_price_max', 'max'), # 판매가격 최대값
        ('sell_price_min', 'min'), # 판매가격 최소값
    ],
    'sell_price_diff': [('sell_price_diff_mean', 'mean')], # 판매가격 변화량 평균값
    'sell_price_lag': [('sell_price_lag_mean', 'mean')], # 1주 전 판매가격 평균값
    'sell_price_rolling': [('sell_price_rolling_mean', 'mean')], # 1달 판매가격 이동평균 평균값
    'sell_price_rolling_diff': [('sell_price_rolling_diff_mean', 'mean')], # 1달 판매가격 이동평균 변화량 평균값
    'sell_price_in_store': [('sell_price_in_store_mean', 'mean')], # 매장 안에서의 상대적 판매 가격 평균값
}

for level_idx, level in tqdm(enumerate(levels, start=1), total=len(levels)):
    if level_idx != 12:
        continue
    agg_cols = level + ['d'] if level else ['d']
    agg_df = data_df.groupby(agg_cols).agg(**{
        new_col: (col, func) 
        for col, aggs in agg_funcs.items() 
        for new_col, func in aggs
    }).reset_index()

    agg_df = agg_df.fillna(0)

    agg_df['sort_key'] = agg_df['d'].str[2:].astype(int)
    agg_df = agg_df.sort_values(['sort_key'] + level)
    agg_df = agg_df.drop(columns=['sort_key']).reset_index(drop=True)
    
    agg_df.insert(0, 'level', level_idx)
    agg_df.to_csv(f'{save_data_dir}/agg_df_level_{level_idx}.csv', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [15:25<00:00, 77.14s/it]
